In [16]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.colors import ListedColormap
import random

### Завдання 1

In [22]:
plt.switch_backend('TkAgg')  

# Початкове поле 30x30 
grid = np.zeros((30, 30), dtype=int)


# Блоки
grid[2:4, 2:4] = 1
grid[5:7, 8:10] = 1
grid[25:27, 25:27] = 1

grid[8:11, 15:18] = [[0,1,0], [1,0,1], [0,1,0]]

# Пульсуючі 
grid[12, 5:8] = 1  
grid[15:18, 12] = 1  

grid[20:22, 20:24] = [[1,1,1,0], [0,1,1,1]]

pulsar = np.zeros((13, 13))
pulsar[2, 4:7] = 1
pulsar[4:7, 2] = 1
pulsar[4:7, 7] = 1
pulsar[7, 4:7] = 1
pulsar[9, 4:7] = 1
pulsar[4:7, 9] = 1
pulsar[4:7, 11] = 1
pulsar[7, 9:12] = 1
grid[8:21, 8:21] = pulsar

# Рухомі 
glider1 = np.array([
    [0,1,0],
    [0,0,1],
    [1,1,1]
])
grid[3:6, 22:25] = glider1

glider2 = np.array([
    [1,0,0],
    [0,1,1],
    [1,1,0]
])
grid[22:25, 3:6] = glider2

lwss = np.array([
    [0,1,1,1,1],
    [1,0,0,0,1],
    [0,0,0,0,1],
    [1,0,0,1,0]
])
grid[18:22, 18:23] = lwss

# Випадкові кластери для додаткової динаміки
np.random.seed(42) 
random_cluster = np.random.choice([0, 1], size=(6, 6), p=[0.7, 0.3])
grid[24:30, 0:6] = random_cluster

# Функція підрахунку сусідів
def count_neighbors(grid, x, y):
    rows, cols = grid.shape
    total = 0
    for i in range(-1, 2):
        for j in range(-1, 2):
            if i == 0 and j == 0:
                continue
            ni, nj = (x + i) % rows, (y + j) % cols
            total += grid[ni, nj]
    return total

#  Оновлення поля
def update(frameNum, img, grid):
    newGrid = grid.copy()
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            neighbors = count_neighbors(grid, i, j)
            if neighbors == 2:
                newGrid[i, j] = grid[i, j]  # не змінюється
            elif neighbors == 3:
                newGrid[i, j] = 1           # стає живою
            else:
                newGrid[i, j] = 0           # помирає
    img.set_data(newGrid)
    grid[:] = newGrid[:]
    return img,

plt.style.use('dark_background')
fig, ax = plt.subplots(figsize=(8, 8))
img = ax.imshow(grid, interpolation='nearest', cmap='gray', vmin=0, vmax=1)
ax.set_title("Гра «Життя»", fontsize=14)
ax.set_xticks([])
ax.set_yticks([])

# Збільшуємо кількість кадрів і інтервал для довшої анімації
anim = animation.FuncAnimation(fig, update, fargs=(img, grid), frames=500, interval=200, blit=False)

plt.show()

### Завдання 2

In [25]:
class RingwormModel:
    def __init__(self, N=21):
        # Ініціалізація моделі з сіткою N x N
        if N % 2 == 0:
            N += 1  # Забезпечуємо непарний розмір для центру
        self.N = N
        self.grid = np.zeros((N, N), dtype=int)  # 0-здорова, 1-інфікована, 2-імунітет
        self.infection_time = np.zeros((N, N), dtype=int)  # час з моменту зараження
        self.center = N // 2
        self.iteration_count = 0
        
        # Початкове зараження в центрі
        self.grid[self.center, self.center] = 1
        self.infection_time[self.center, self.center] = 1
        
        # Статистика для графіків
        self.stats = {
            'healthy': [N*N - 1],  # початково всі здорові крім центру
            'infected': [1],       # одна інфікована клітина
            'immune': [0]          # немає клітин з імунітетом
        }
        
    def get_cell_state(self, time_since_infection):
        # Визначення стану клітини на основі часу зараження
        if time_since_infection == 0:
            return 0  # здорова
        elif time_since_infection <= 6:
            return 1  # інфікована
        elif time_since_infection <= 10:
            return 2  # несприйнятлива (імунітет)
        else:
            return 0  # знову здорова
    
    def update(self):
        # Оновлення стану всіх клітин за одну ітерацію
        if self.iteration_count >= 50:
            return self_grid
            
        new_grid = self.grid.copy()
        new_infection_time = self.infection_time.copy()
        
        # Оновлення часу зараження та стану клітин
        for i in range(self.N):
            for j in range(self.N):
                if self.infection_time[i, j] > 0:
                    new_infection_time[i, j] = self.infection_time[i, j] + 1
                    new_grid[i, j] = self.get_cell_state(new_infection_time[i, j])
        
        # Поширення інфекції на сусідні клітини
        for i in range(self.N):
            for j in range(self.N):
                if self.grid[i, j] == 1:  # Якщо клітина інфікована
                    # Перевіряємо всіх 8 сусідів
                    for di in [-1, 0, 1]:
                        for dj in [-1, 0, 1]:
                            if di == 0 and dj == 0:
                                continue  # Пропускаємо поточну клітину
                            ni, nj = i + di, j + dj
                            # Перевірка меж сітки
                            if 0 <= ni < self.N and 0 <= nj < self.N:
                                # Заражаємо здорову клітину з ймовірністю 50%
                                if self.grid[ni, nj] == 0 and random.random() < 0.5:
                                    new_grid[ni, nj] = 1
                                    new_infection_time[ni, nj] = 1
        
        self.grid = new_grid
        self.infection_time = new_infection_time
        self.iteration_count += 1
        
        # Оновлення статистики
        healthy = np.sum(self.grid == 0)
        infected = np.sum(self.grid == 1)
        immune = np.sum(self.grid == 2)
        
        self.stats['healthy'].append(healthy)
        self.stats['infected'].append(infected)
        self.stats['immune'].append(immune)
        
        return self.grid
    
    def get_infection_probability_analysis(self):
        # Розрахунок ймовірностей зараження в залежності від кількості інфікованих сусідів
        probabilities = {}
        
        for infected_neighbors in range(9):
            if infected_neighbors == 0:
                prob = 0.0
            else:
                # Ймовірність = 1 - (ймовірність не заразитися жодним сусідом)
                prob = 1 - (0.5 ** infected_neighbors)
            probabilities[infected_neighbors] = prob
        
        return probabilities

def simulate_ringworm():
    # Основна функція симуляції з анімацією
    model = RingwormModel(N=21)
    
    # Створення вікна з двома графіками
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Кольорова схема: зелений-здоровий, червоний-інфікований, синій-імунітет
    cmap = ListedColormap(['green', 'red', 'blue'])
    
    # Графік сітки станів
    img = ax1.imshow(model.grid, cmap=cmap, vmin=0, vmax=2, interpolation='nearest')
    ax1.set_title('Поширення стригучого лишая (Ітерація 0/50)')
    ax1.set_xticks([])
    ax1.set_yticks([])
    
    # Графік динаміки популяції
    time_points = list(range(len(model.stats['healthy'])))
    line1, = ax2.plot(time_points, model.stats['healthy'], 'g-', label='Здорові (S)')
    line2, = ax2.plot(time_points, model.stats['infected'], 'r-', label='Інфіковані (K)')
    line3, = ax2.plot(time_points, model.stats['immune'], 'b-', label='Імунітет (L)')
    
    ax2.set_xlabel('Ітерації (t)')
    ax2.set_ylabel('Кількість клітин')
    ax2.set_title('Динаміка поширення інфекції')
    ax2.legend()
    ax2.grid(True)
    
    def animate(frame):
        # Функція анімації для оновлення графіків
        if model.iteration_count < 50:
            model.update()
            img.set_data(model.grid)
            
            # Оновлення графіка статистики
            time_points = list(range(len(model.stats['healthy'])))
            line1.set_data(time_points, model.stats['healthy'])
            line2.set_data(time_points, model.stats['infected'])
            line3.set_data(time_points, model.stats['immune'])
            ax2.set_xlim(0, len(time_points)-1)
            ax2.set_ylim(0, model.N * model.N)
            
            ax1.set_title(f'Поширення стригучого лишая (Ітерація {model.iteration_count}/50)')
        else:
            # Зупинка анімації після 50 ітерацій
            anim.event_source.stop()
            ax1.set_title('Поширення стригучого лишая (ЗАВЕРШЕНО - 50/50 ітерацій)')
        
        return img, line1, line2, line3
    
    # Створення анімації
    anim = animation.FuncAnimation(fig, animate, frames=51, interval=500, blit=False, repeat=False)
    plt.tight_layout()
    plt.show()
    
    return model, anim

def print_probability_table():
    # Виведення таблиці ймовірностей зараження
    print("Таблиця ймовірностей ураження центральної клітини")
    print("Кількість інфікованих сусідів  | Ймовірність ураження")
    print("-" * 60)
    
    model = RingwormModel()
    probs = model.get_infection_probability_analysis()
    
    for neighbors, prob in probs.items():
        print(f"{neighbors:^30} | {prob:.4f}")

def plot_final_statistics():
    # Побудова фінальних графіків після завершення симуляції
    model = RingwormModel(N=21)
    
    # Запуск симуляції на 50 ітерацій
    for i in range(50):
        model.update()
    
    # Створення графіків результатів
    plt.figure(figsize=(12, 5))
    
    # Графік 1: Динаміка змін популяції
    plt.subplot(1, 2, 1)
    time_points = list(range(len(model.stats['healthy'])))
    
    plt.plot(time_points, model.stats['healthy'], 'g-', linewidth=2, label='Здорові S(t)')
    plt.plot(time_points, model.stats['infected'], 'r-', linewidth=2, label='Інфіковані K(t)')
    plt.plot(time_points, model.stats['immune'], 'b-', linewidth=2, label='Імунітет L(t)')
    plt.xlabel('Ітерації (t)')
    plt.ylabel('Кількість клітин')
    plt.title('Динаміка поширення інфекції')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Графік 2: Ймовірності зараження
    plt.subplot(1, 2, 2)
    probs = model.get_infection_probability_analysis()
    bars = plt.bar(probs.keys(), probs.values(), color='orange', alpha=0.7, edgecolor='darkorange')
    plt.xlabel('Кількість інфікованих сусідів')
    plt.ylabel('Ймовірність ураження')
    plt.title('Ймовірність ураження центральної клітини')
    plt.grid(True, alpha=0.3)
    
    # Додавання числових значень на стовпці
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    # Виведення фінальної статистики
    total_cells = model.N * model.N
    final_healthy = model.stats['healthy'][-1]
    final_infected = model.stats['infected'][-1]
    final_immune = model.stats['immune'][-1]
    
    print(f"Загальна кількість клітин: {total_cells}")
    print(f"Здорові клітини: {final_healthy} ({final_healthy/total_cells*100:.1f}%)")
    print(f"Інфіковані клітини: {final_infected} ({final_infected/total_cells*100:.1f}%)")
    print(f"Клітини з імунітетом: {final_immune} ({final_immune/total_cells*100:.1f}%)")

if __name__ == "__main__":
    # Головна програма
    print_probability_table()
    print()
    model, anim = simulate_ringworm()
    plot_final_statistics()

Таблиця ймовірностей ураження центральної клітини
Кількість інфікованих сусідів  | Ймовірність ураження
------------------------------------------------------------
              0                | 0.0000
              1                | 0.5000
              2                | 0.7500
              3                | 0.8750
              4                | 0.9375
              5                | 0.9688
              6                | 0.9844
              7                | 0.9922
              8                | 0.9961

Загальна кількість клітин: 441
Здорові клітини: 441 (100.0%)
Інфіковані клітини: 0 (0.0%)
Клітини з імунітетом: 0 (0.0%)


### Завдання 3

In [20]:
# Параметри моделі 
N = 100             # розмір сітки
p_grow = 0.01       # ймовірність росту нового дерева
p_lightning = 0.000005  # ймовірність самозаймання дерева
frames = 300        # к-сть ітерацій

# Стан клітин 
EMPTY = 0
BURNING = 1
FOREST = 2

# Ініціалізація поля 
grid = np.random.choice([EMPTY, FOREST], size=(N, N), p=[0.4, 0.6])

# Функція підрахунку 4 сусідів
def neighbors_4(x, y):
    return [
        ((x - 1) % N, y),
        ((x + 1) % N, y),
        (x, (y - 1) % N),
        (x, (y + 1) % N)
    ]

# Оновлення стану
def update(frameNum, img, grid):
    newGrid = grid.copy()
    for i in range(N):
        for j in range(N):
            if grid[i, j] == FOREST:
                # Якщо хоч один сусід горить → загоряється
                if any(grid[x, y] == BURNING for x, y in neighbors_4(i, j)):
                    newGrid[i, j] = BURNING
                # Інакше — може самозайнятися
                elif random.random() < p_lightning:
                    newGrid[i, j] = BURNING

            elif grid[i, j] == BURNING:
                # Згорає → порожнє місце
                newGrid[i, j] = EMPTY

            elif grid[i, j] == EMPTY:
                # Порожня клітина може вирости у ліс
                if random.random() < p_grow:
                    newGrid[i, j] = FOREST

    img.set_data(newGrid)
    grid[:] = newGrid[:]
    return img,

plt.switch_backend('TkAgg')
from matplotlib.colors import ListedColormap
cmap = ListedColormap(['black', 'red', 'green'])

fig, ax = plt.subplots(figsize=(6, 6))
img = ax.imshow(grid, cmap=cmap, vmin=0, vmax=2)
ax.set_title("Модель розповсюдження лісової пожежі", fontsize=14)
ax.set_xticks([])
ax.set_yticks([])

anim = animation.FuncAnimation(
    fig, update, fargs=(img, grid),
    frames=frames, interval=100, blit=False
)
plt.show()
